In [ ]:
import pandas
import seaborn
import numpy
from matplotlib import pyplot
import warnings

warnings.filterwarnings('ignore')

## t-test
모집단의 분산과 표준편차를 알지 못할 때 사용되는 통계적 검정 방법으로, 표본에서 추정된 분산이나 표준편차를 활용하여 검정한다.


## t-test를 위한 가정
1. 종속변수는 연속형(양적, Continuous) 변수여야 한다.
2. 독립변수는 이산형/범주형 변수여야 한다.
3. 모집단의 분산과 표준편차를 알 수 없다.
4. 모집단의 분포는 정규분포를 따른다.
5. 등분산성의 가정이 충족되어야 한다.

## 귀무가설(H₀), 대립가설(H1)
1. 귀무가설(H0) : 영가설, A와 B는 차이가 없다.  결과는 우연히 일어난 것이다.
2. 대립가설(H1) : A와 B는 차이가 있다.

In [ ]:
table = pandas.DataFrame([['N', 'n'],
                          ['µ', 'x'],
                          ['σ2', 's2'],
                          ['σ', 's']],
                         columns = ['모집단(population)', '표본(sample)'],
                         index = ['관측치', '평균값', '분산', '편차'])
table

In [ ]:
car = pandas.read_csv('auto-mpg.csv')
car.head()

In [ ]:
car.info()

In [ ]:
origin_usa = car[car['origin'] == 1]['mpg']
origin_eu  = car[car['origin'] == 2]['mpg']
origin_jpn = car[car['origin'] == 3]['mpg']
origin_list = [origin_usa, origin_eu, origin_jpn]

pyplot.style.use('seaborn-whitegrid')
pyplot.figure(figsize = (10, 4))
pyplot.boxplot(origin_list, labels = ['usa', 'eu', 'jpn'])
pyplot.show()

In [ ]:
seaborn.set_style('whitegrid')
figure = pyplot.figure(figsize = (10, 4))
ax1 = figure.add_subplot(1, 2, 1)
ax2 = figure.add_subplot(1, 2, 2)
seaborn.stripplot(data = car, hue = 'origin', x = 'origin', y = 'mpg', ax = ax1)
seaborn.swarmplot(data = car, hue = 'origin', x = 'origin', y = 'mpg', ax = ax2)
pyplot.show()

##  단일표본(one-sample) t-test
1. 단일 모집단에 대한 가설 검정
2. 단일 모집단에서 관심이 있는 연속형 변수(Columns)의 평균값을 특정 기준값과 비교할 때 사용
3. 모집단이 정규분포를 따른다는 가정하에 검정통계량(statistic)을 계산
4. 문제에 따라  양측검정(two-sided), 단측검정: 우측검정(greater), 좌측검정(less)
  - 양측검정(two-sided)
    - H0: 평균 ＝ 기준값
    - H1: 평균 ≠ 기준값
  - 우측검정(greater)
    - H0: 평균 ＝ 기준값
    - H1: 평균 > 기준값
  - 좌측검정(less)
    - H0: 평균 ＝ 기준값
    - H1: 평균 < 기준값

In [ ]:
from IPython.display import Image
Image(filename = 'one_sample_t_test_fomula.png')

In [ ]:
mean_whole = car['mpg'].mean()
mean_usa = car[car['origin'] == 1]['mpg'].mean()
print('전체평균:', mean_whole)
print('미국평균:', mean_usa)

In [ ]:
mask_usa = car['origin'] == 1
var_usa = car[mask_usa]['mpg'].var()
std_usa = car[mask_usa]['mpg'].std()
count_usa = len(car[mask_usa])
print('표준분산:', var_usa)
print('표준편차:', std_usa)
print('표본수량:', count_usa)

In [ ]:
# 표본분산, 표본편차 구하는 방법
mask_usa = car['origin'] == 1
sr_usa = car[mask_usa]['mpg']

# df(degrees of freedom)
df_usa = count_usa - 1
var_usa = ((sr_usa - mean_usa) ** 2).sum() / df_usa
std_usa = var_usa ** 0.5
print('표준분산:', var_usa)
print('표준편차:', std_usa)
print('표본수량:', count_usa)

In [ ]:
# 통계량
statistic = (mean_usa - mean_whole) / (std_usa / (count_usa ** 0.5))
print('통계량:', statistic)
print('자유도:', df_usa)

In [ ]:
from scipy import stats

# 단일표본(one-sample) t-test : scipy.stats.ttest_1samp
# scipy.stats.ttest_1samp(Series[배열], 비교값,
#                         alternative = 'two-sided'/'greater'/'less',
#                         nan_policy = 'propagate'/'raise'/'omit',
#                         random_state = None)
mean_whole = car['mpg'].mean()
print('전체평균:', mean_whole, end = '\n\n')
mpg_usa = car[car['origin'] == 1]['mpg']
result = stats.ttest_1samp(mpg_usa, mean_whole, alternative = 'two-sided')
print(result)
print('statistic(통계량):', result.statistic)
print('p-value(p-값):', result.pvalue)
print('df(자유도):', result.df, end = '\n\n')
print('유의확율   5% 미만:', result.pvalue < 0.05)
print('유의확율 0.1% 미만:', result.pvalue < 0.001)
print('유의확율 0.000000001% 미만:', result.pvalue < 0.00000000001)

In [ ]:
from scipy import stats

result = stats.wilcoxon(mpg_usa - mean_whole)
print(result)
print('statistic(통계량):', result.statistic)
print('p-value(p-값):', result.pvalue)
print('유의확율   5% 미만:', result.pvalue < 0.05)
print('유의확율 0.1% 미만:', result.pvalue < 0.001)

##  독립표본(independent-sample, two-sample) t-test
1. 두개의 모집단에 대한 가설 검정
2. 두개의 모집단의 평균을 비교하고자 할 때 사용
3. 가정: 정규성, 독립성, 등분산성
4. 문제에 따라  양측검정(two-sided), 단측검정: 우측검정(greater), 좌측검정(less)
  - 양측검정(two-sided)
    - H0: 집단1_평균 ＝ 집단2_평균
    - H1: 집단1_평균 ≠ 집단2_평균
  - 우측검정(greater)
    - H0: 집단1_평균 ＝ 집단2_평균
    - H1: 집단1_평균 > 집단2_평균
  - 좌측검정(less)
    - H0: 집단1_평균 ＝ 집단2_평균
    - H1: 집단1_평균 < 집단2_평균

In [ ]:
from IPython.display import Image
Image(filename = 'independent_sample_t_test_fomula.png')

In [ ]:
import seaborn
from scipy import stats

# 2개의 샘플의 정규성 여부를 그래프로 확인
mpg_usa = car[car['origin'] == 1]['mpg']
mpg_jpn = car[car['origin'] == 3]['mpg']

figure = pyplot.figure(figsize = (9, 6))
ax1 = figure.add_subplot(2, 1, 1)
ax2 = figure.add_subplot(2, 1, 2)
seaborn.distplot(mpg_usa, kde = True, hist = True, fit = stats.norm, ax = ax1)
seaborn.distplot(mpg_jpn, kde = True, hist = True, fit = stats.norm, ax = ax2)
pyplot.show()

In [ ]:
from scipy import stats

mpg_usa = car[car['origin'] == 1]['mpg']
mpg_jpn = car[car['origin'] == 3]['mpg']

# 2개의 샘플의 정규성 여부를 확인

# scipy.stats.shapiro(배열)        : Shapiro-Wilk Test, 데이터가 정규분포를 따르는지 검정
#                                                  데이터수가 5000개미만일 때 사용한다.
# scipy.stats.kstest(배열1, 배열2) : Kolmogorove-Smirnov Test, 2개 그룹의 분포가 동일한지 검정
# scipy.stats.kstest(배열, 'norm') : Kolmogorove-Smirnov Test, 데이터가 정규분포를 따르는지 검정
#numpy.random.seed(0)
print(stats.shapiro(mpg_usa))
print(stats.kstest(mpg_usa, 'norm'), end = '\n\n')
print(stats.shapiro(mpg_jpn))
print(stats.kstest(mpg_jpn, 'norm'), end = '\n\n')
print('미국 정규성 유의확률 5%미만:', stats.shapiro(mpg_usa).pvalue < 0.05)
print('일본 정규성 유의확률 5%미만:', stats.shapiro(mpg_jpn).pvalue < 0.05)

## 정규성 검정
- 정규성 검정은 데이터가 정규분포를 따르는지 검정하는 것을 의미한다.
- (중심극한정리) 보통 표본이 30개이상인 경우 정규성을 만족한다고 가정합니다.
- 하지만, 표본이 30개 이상이어도 데이터 특성에 의해 정규분를 따르지 않을 수도 있기에 정규성을 진행한다.

In [ ]:
import seaborn

# 2개의 샘플의 정규성 여부를 그래프로 확인
figure = pyplot.figure(figsize = (9, 6))
ax1 = figure.add_subplot(2, 1, 1)
ax2 = figure.add_subplot(2, 1, 2)
seaborn.distplot(mpg_usa, kde = True, hist = True, ax = ax1)
seaborn.distplot(mpg_jpn, kde = True, hist = True, ax = ax1)
seaborn.distplot(mpg_usa - mpg_usa.mean(), kde = True, hist = True, ax = ax2)
seaborn.distplot(mpg_jpn - mpg_jpn.mean(), kde = True, hist = True, ax = ax2)
ax1.lines[0].set_linestyle(':')
ax2.lines[0].set_linestyle(':')
pyplot.show()

In [ ]:
from scipy import stats

# 2개의 샘플의 등분산성 확인
# scipy.stats.kstest(배열1, 배열2) : Kolmogorove-Smirnov Test, 2개 그룹의 분포가 동일한지 검정
# scipy.stats.kstest(배열, 'norm') : Kolmogorove-Smirnov Test, 데이터가 정규분포를 따르는지 검정
result = stats.kstest(mpg_usa, mpg_jpn)
print(result)
print('statistic(통계량):', result.statistic)
print('p-value(p-값):', result.pvalue)
print('유의확율   5% 미만:', result.pvalue < 0.05)
print('유의확율 0.1% 미만:', result.pvalue < 0.001)

In [3]:
from scipy import stats

# 2개의 샘플의 등분산성 확인
result_bartlett = stats.bartlett(mpg_usa, mpg_jpn)
result_levene   = stats.levene(mpg_usa, mpg_jpn)

print('바트렛 검정 - 유의확율   5% 미만:', result_bartlett.pvalue < 0.05)
print('바트렛 검정 - 유의확율 0.1% 미만:', result_bartlett.pvalue < 0.001, end = '\n\n')
print('레빈 검정 - 유의확율   5% 미만:', result_levene.pvalue < 0.05)
print('레빈 검정 - 유의확율 0.1% 미만:', result_levene.pvalue < 0.001)

바트렛 검정 - 유의확율     5% 미만: False
바트렛 검정 - 유의확율 0.001% 미만: False

레빈 검정 - 유의확율     5% 미만: False
레빈 검정 - 유의확율 0.001% 미만: False


In [ ]:
from scipy import stats

# 독립표본(independent-sample, two-sample) t-test : scipy.stats.ttest_1samp
# scipy.stats.ttest_ind(Series[배열1], Series[배열2], equal_var = True/False,
#                       alternative = 'two-sided'/'greater'/'less',
#                       nan_policy = 'propagate'/'raise'/'omit',
#                       random_state = None)
result = stats.ttest_ind(mpg_usa, mpg_jpn, equal_var = False, alternative = 'less')
print(result)
print('statistic(통계량):', result.statistic)
print('p-value(p-값):', result.pvalue)
print('유의확율   5% 미만:', result.pvalue < 0.05)
print('유의확율 0.1% 미만:', result.pvalue < 0.001)

In [ ]:
import statsmodels.stats.weightstats

# statsmodels.stats.weightstats.ttest_ind(Series[배열1], Series[배열2], usevar = 'pooled'/unequal',
#                                         alternative = 'two-sided'/'larger'/'smaller',
#                                         weights = (None, None))
result = statsmodels.stats.weightstats.ttest_ind(mpg_usa, mpg_jpn, usevar = 'unequal',
                                                 alternative = 'smaller', weights = (None, None))
print(result)
print('statistic(통계량):', result[0])
print('p-value(p-값):', result[1])
print('유의확율   5% 미만:', result[1] < 0.05)
print('유의확율 0.1% 미만:', result[1] < 0.001)

##  대응표본(two-sample paired) t-test
1. 단일 모집단에 대한 가설 검정
2. 단일 모집단에 대해 2번의 처리를 가하고, 2개의 처리에 대한 평균 차이을 비교하고자 할 때 사용
3. 모집단의 관측값이 정규분포를 따른다는 가저을 만족해야함 (정규성 가정)
4. 문제에 따라  양측검정(two-sided), 단측검정: 우측검정(greater), 좌측검정(less)
  - 양측검정(two-sided)
    - H0: 집단_처리1 ＝ 집단_처리2
    - H1: 집단_처리1 ≠ 집단_처리2
  - 우측검정(greater)
    - H0: 집단_처리1 ＝ 집단_처리2
    - H1: 집단_처리1 > 집단_처리2
  - 좌측검정(less)
    - H0: 집단_처리1 ＝ 집단_처리2
    - H1: 집단_처리1 < 집단_처리2

In [ ]:
blood = pandas.read_csv('blood_pressure.csv', index_col = 'no')
blood

In [ ]:
before = blood['before']
after  = blood['after']

print(stats.shapiro(before))
print(stats.shapiro(after))
print('before 정규성 유의확률 5%미만:', stats.shapiro(before).pvalue < 0.05)
print('after  정규성 유의확률 5%미만:', stats.shapiro(after).pvalue < 0.05)

In [ ]:
# 2개의 샘플의 정규성 여부를 그래프로 확인
figure = pyplot.figure(figsize = (9, 6))
ax1 = figure.add_subplot(2, 1, 1)
ax2 = figure.add_subplot(2, 1, 2)
seaborn.distplot(before, kde = True, hist = True, ax = ax1)
seaborn.distplot(after , kde = True, hist = True, ax = ax1)
seaborn.distplot(before - before.mean(), kde = True, hist = True, ax = ax2)
seaborn.distplot(after  - after.mean() , kde = True, hist = True, ax = ax2)
ax1.lines[0].set_linestyle(':')
ax2.lines[0].set_linestyle(':')
pyplot.show()

In [ ]:
from scipy import stats

# 2개의 샘플의 등분산성 확인
result = stats.kstest(before, after)
print(result)
print('statistic(통계량):', result.statistic)
print('p-value(p-값):', result.pvalue)
print('유의확율   5% 미만:', result.pvalue < 0.05)
print('유의확율 0.1% 미만:', result.pvalue < 0.001)

In [ ]:
# 2개의 샘플의 등분산성 확인
result = stats.f_oneway(before, after)
print(result)
print('statistic(통계량):', result.statistic)
print('p-value(p-값):', result.pvalue)
print('유의확율   5% 미만:', result.pvalue < 0.05)
print('유의확율 0.1% 미만:', result.pvalue < 0.001)

In [ ]:
from scipy import stats

# scipy.stats.ttest_rel(Series[배열1], Series[배열2],
#                       alternative = 'two-sided'/'greater'/'less',
#                       nan_policy = 'propagate'/'raise'/'omit')
result = stats.ttest_rel(blood['before'], blood['after'], alternative = 'greater')
print(result)
print('statistic(통계량):', result[0])
print('p-value(p-값):', result[1])
print('유의확율   5% 미만:', result[1] < 0.05)
print('유의확율 0.1% 미만:', result[1] < 0.001)

In [ ]:
import statsmodels.stats.weightstats

# statsmodels.stats.weightstats.ttost_paired(Series[배열1], Series[배열2], low, upp,
#                                            transform = None, weights = None)
result = statsmodels.stats.weightstats.ttost_paired(blood['before'], blood['after'],
                                                    0, 14, transform = None, weights = None)
print(result)
print('statistic(통계량):', result[1][0])
print('p-value(p-값):', result[0])
print('유의확율   5% 미만:', result[0] < 0.05)
print('유의확율 0.1% 미만:', result[0] < 0.001)

# scipy.stats 모듈